In [ ]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns
from random import shuffle
from tqdm import tqdm
# import scipy.stats
from scipy import stats
import random
from collections import Counter
from scipy.optimize import curve_fit
import multiprocessing
from numpy.random import RandomState

import random
# plt.style.use('fivethirtyeight')
%matplotlib inline

# plt.style.available


# Создание агентов

In [ ]:
data_age = pd.read_excel('./data_age.xlsx')
data_age


In [ ]:
print(data_age["Men"].sum(), data_age["Women"].sum(),
      data_age["Men"].sum() + data_age["Women"].sum())


In [ ]:
percentage = data_age.copy()


In [ ]:
percentage["Men"] = percentage['Men'] / percentage['Men'].sum()
percentage["Women"] = percentage['Women'] / percentage['Women'].sum()


In [ ]:
percentage


In [ ]:
men_cnt = int(data_age["Men"].sum() * 12000 /
              (data_age["Men"].sum() + data_age["Women"].sum()))
women_cnt = int(data_age["Women"].sum() * 12000 /
                (data_age["Men"].sum() + data_age["Women"].sum()))


In [ ]:
percentage["Men"] = percentage["Men"] * men_cnt
percentage["Women"] = percentage["Women"] * women_cnt


In [ ]:
percentage["Men"] = percentage.Men.round().astype(int)
percentage["Women"] = percentage.Women.round().astype(int)


In [ ]:
percentage


In [ ]:
men = percentage["Men"].to_numpy()
women = percentage["Women"].to_numpy()
age = percentage["Age"].to_numpy()


In [ ]:
men_column = np.repeat(age, men)
women_column = np.repeat(age, women)

men_cnt = percentage["Men"].sum()
women_cnt = percentage["Women"].sum()


In [ ]:
men_pd = pd.DataFrame()
women_pd = pd.DataFrame()


In [ ]:
men_pd["Age"] = men_column
men_pd["Sex"] = ["M"] * men_cnt
women_pd["Age"] = women_column
women_pd["Sex"] = ["W"] * women_cnt


In [ ]:
base = pd.concat([men_pd, women_pd], ignore_index=True)


In [ ]:
base["Id"] = np.arange(men_cnt + women_cnt)


In [ ]:
base = base[["Id", "Age", "Sex"]]


In [ ]:
base


In [ ]:
marriage_prct = (21383475 + 21794632) / (7590837 + 2294647 + 984164 +
                                         1414882 + 6578532 + 4737595 + 1400283 + 6724715 + 21383475 + 21794632)
marriage_prct


In [ ]:
data_marriage = pd.read_excel('./data_marriage.xlsx')
data_marriage = data_marriage.drop(columns=[
                                   'not indicated', 'in registered marriage', 'in non-registered marriage', 'Total', 'state of marriage indicated'])
data_marriage


In [ ]:
df_m = data_marriage.iloc[:15, :]
df_w = data_marriage.iloc[15:, :]
df_m = df_m.drop([0])
df_w = df_w.drop([15])
df_m = df_m.reset_index(drop=True)
df_w = df_w.reset_index(drop=True)


In [ ]:
df_m["not marriaged"] = df_m.iloc[:, 2:].sum(axis=1)
df_m = df_m[["Городское население", "marriaged", "not marriaged"]]
df_w["not marriaged"] = df_w.iloc[:, 2:].sum(axis=1)
df_w = df_w[["Городское население", "marriaged", "not marriaged"]]


In [ ]:
df_m


In [ ]:
df_w


In [ ]:
df_m["marriage percent"] = df_m["marriaged"] / \
    (df_m["marriaged"] + df_m["not marriaged"])
df_m = df_m.drop(columns=["marriaged", "not marriaged"])
df_w["marriage percent"] = df_w["marriaged"] / \
    (df_w["marriaged"] + df_w["not marriaged"])
df_w = df_w.drop(columns=["marriaged", "not marriaged"])


In [ ]:
df_m


In [ ]:
df_w


In [ ]:
list_of_slices = [(16, 17), (18, 19), (20, 24), (25, 29), (30, 34), (35, 39),
                  (40, 44), (45, 49), (50, 54), (55, 59), (60, 64), (65, 69), (70, 74), (75, 100)]


In [ ]:
men_age_range = []
women_age_range = []

for slices in list_of_slices:
    men_age_range += [base[(base["Sex"] == "M") & (base["Age"]
                                                   >= slices[0]) & (base["Age"] <= slices[1])]["Id"].count()]
    women_age_range += [base[(base["Sex"] == "W") & (base["Age"]
                                                     >= slices[0]) & (base["Age"] <= slices[1])]["Id"].count()]


In [ ]:
df_m["marriage percent"] = round(
    df_m["marriage percent"] * men_age_range).astype(int)


In [ ]:
df_w["marriage percent"] = round(
    df_w["marriage percent"] * women_age_range).astype(int)


In [ ]:
df_m


In [ ]:
df_w


In [ ]:
minimums = np.minimum(df_w["marriage percent"].to_numpy(),
                      df_m["marriage percent"].to_numpy())
minimums[-1] = 86
minimums[4] = 316


In [ ]:
married_in_range = np.repeat((minimums[2:13] / 5).astype(int), 5)
married_in_range = np.concatenate((np.array([0]), np.array([1]), married_in_range, np.array(
    [10] * 6), np.array([4] * 9), np.array([0] * 6), np.array([0] * 3)))
len(married_in_range)


In [ ]:
tmp = base.copy()
tmp["Married"] = 0
tmp


In [ ]:
for age, cnt in zip(range(16, 99), married_in_range):
    tmp.loc[tmp['Sex'].isin(["M"]) & tmp['Age'].isin([age]), 'Married'] = np.array(
        [1] * cnt + [0] * (len(tmp[(tmp['Sex'] == "M") & (tmp['Age'] == age)]) - cnt)).astype(int)
    tmp.loc[tmp['Sex'].isin(["W"]) & tmp['Age'].isin([age]), 'Married'] = np.array(
        [1] * cnt + [0] * (len(tmp[(tmp['Sex'] == "W") & (tmp['Age'] == age)]) - cnt)).astype(int)


In [ ]:
base_final = tmp.copy()


In [ ]:
base_final.to_csv("./final_base.csv", index=None)


# Графики половозрастное

In [ ]:
data_marriage = pd.read_excel('./data_marriage.xlsx')
data_age = pd.read_excel('./data_age.xlsx')
data_age.at[0, 'Age'] = '0'
data_age.at[120, "Age"] = "100"
data_age = data_age[data_age["Age"].apply(
    lambda x: ("-" not in str(x)) and ("–" not in str(x)))]
data_age = data_age.reset_index(drop=True)
data_age['Age'] = data_age['Age'].astype('int')


In [ ]:
base_final = pd.read_csv("./final_base.csv")


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14, 10), dpi=600)

x1 = np.arange(96)
y1 = base_final[base_final["Sex"] == "M"].groupby(by=["Age"]).count()[
    "Id"].to_list()
x2 = np.arange(99)
y2 = base_final[base_final["Sex"] == "W"].groupby(by=["Age"]).count()[
    "Id"].to_list()


x3 = data_age['Age']
y3 = data_age['Men']
x4 = data_age['Age']
y4 = data_age['Women']

axes[0].plot(x1, y1, 'o')
axes[0].plot(x2, y2, 'o')
axes[1].plot(x3, y3, 'o')
axes[1].plot(x4, y4, 'o')

# fig.suptitle('Количество людей в зависимости от возраста', fontsize=25)

axes[0].set_title('Кластер', pad=10, fontsize=16)
axes[1].set_title('Москва', pad=10, fontsize=16)


axes[0].set_ylabel('Количество', labelpad=15, fontsize=11)
axes[0].set_xlabel('Возраст', fontsize=11)
axes[1].set_ylabel('Количество', labelpad=15, fontsize=11)
axes[1].set_xlabel('Возраст', fontsize=11)

axes[0].set_facecolor('#f4f4f4')
axes[1].set_facecolor('#f4f4f4')
axes[0].grid(color='white')
axes[1].grid(color='white')

axes[1].tick_params('x', labelsize=11)
axes[1].tick_params('y', labelsize=11)
axes[0].tick_params('x', labelsize=11)
axes[0].tick_params('y', labelsize=11)

fig.tight_layout()

axes[0].legend(['Мужчины', 'Женщины'], loc='upper right', fontsize=11)
axes[1].legend(['Мужчины', 'Женщины'], loc='upper right', fontsize=11)

fig.savefig('new_claster.png', dpi=fig.dpi)


In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(np.arange(71), base_final[base_final["Married"] == 1].groupby(
    by=["Age"]).count()["Id"].to_list())
plt.title('Количество женатых людей в зависимости от возраста')
plt.xlabel('Возраст')
plt.ylabel('Количество')
# plt.legend();


# Создание пар, знакомств и атрибутов агентов

In [ ]:
base_final = pd.read_csv("./final_base.csv")
base_final["TB"] = ["S"] * 11996
base_final


In [ ]:
first = np.full((base_final.shape[0], 5), -1)
second = np.full((base_final.shape[0], 15), -1)
third = np.full((base_final.shape[0], 50), -1)


In [ ]:
list_of_slices = [(16, 17), (18, 19), (20, 24), (25, 29), (30, 34), (35, 39),
                  (40, 44), (45, 49), (50, 54), (55, 59), (60, 64), (65, 69), (70, 74), (75, 100)]

for age in list_of_slices:
    men = base_final[(base_final["Married"] == 1) & (base_final["Age"] >= age[0]) & (
        base_final["Age"] <= age[1]) & (base_final["Sex"] == "M")]["Id"].to_numpy()
    women = base_final[(base_final["Married"] == 1) & (base_final["Age"] >= age[0]) & (
        base_final["Age"] <= age[1]) & (base_final["Sex"] == "W")]["Id"].to_numpy()
    np.random.shuffle(men)
    np.random.shuffle(women)

    first[men, 0] = women.copy()
    first[women, 0] = men.copy()


In [ ]:
babie_ages = [(0, 0), (1, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9),
              (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18)]
parent_ages = [(20, 24, 1), (25, 29, 1), (25, 28, 2), (30, 31, 1), (32, 33, 1), (34, 35, 1), (34, 35, 2), (36, 37, 1), (36, 37, 2),
               (38, 39, 1), (38, 39, 2), (40, 41, 1), (42, 43, 1), (44, 45, 1), (46, 47, 1), (48, 50, 1), (46, 47, 2), (48, 50, 2)]


In [ ]:
for babie, parent in zip(babie_ages, parent_ages):
    babies = base_final[(base_final["Age"] == babie[0]) | (
        base_final["Age"] == babie[1])]["Id"].to_numpy()
    men = base_final[(base_final["Age"] >= parent[0]) & (base_final["Age"] <= parent[1]) & (
        base_final["Married"] == 1) & (base_final["Sex"] == "M")]["Id"].to_numpy()[0:len(babies)]

    women = first[men, 0].astype(int)
    women = women[women != 0].astype(int)

    np.random.shuffle(babies)

    first[men, parent[2]] = babies.copy()
    first[women, parent[2]] = babies.copy()
    first[babies, 3] = men.copy()
    first[babies, 4] = women.copy()


In [ ]:
parent_ages = [(19, 19), (20, 21), (22, 22), (23, 24),
               (25, 26), (27, 28), (29, 30)]
grandpa_ages = [(51, 52, 1), (53, 55, 1), (56, 57, 1),
                (58, 60, 1), (61, 63, 1), (64, 66, 1), (67, 72, 1)]


In [ ]:
for babie, parent in zip(parent_ages, grandpa_ages):
    men = base_final[(base_final["Age"] >= parent[0]) & (base_final["Age"] <= parent[1]) & (
        base_final["Married"] == 1) & (base_final["Sex"] == "M")]["Id"].to_numpy()
    babies = base_final[(base_final["Age"] >= babie[0]) & (
        base_final["Age"] <= babie[1])]["Id"].to_numpy()[0:len(men)]

    women = first[men, 0].astype(int)
    women = women[women != 0].astype(int)

    np.random.shuffle(babies)

    first[men, parent[2]] = babies.copy()
    first[women, parent[2]] = babies.copy()
    first[babies, 3] = men.copy()
    first[babies, 4] = women.copy()


In [ ]:
for age in tqdm(range(7, 19)):
    ppl = base_final[base_final["Age"] == age]["Id"].to_list()
    ppl_50 = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 2)]["Id"].to_list()

    for person in ppl:
        second[person] = (np.random.choice(ppl[:ppl.index(
            person)] + ppl[ppl.index(person) + 1:], 15, replace=False)).astype(int)
    for person in ppl_50:
        third[person] = (np.random.choice(ppl_50[:ppl_50.index(
            person)] + ppl_50[ppl_50.index(person) + 1:], 50, replace=False)).astype(int)


In [ ]:
for age in tqdm(range(19, 25)):
    ppl = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 5)]["Id"].to_list()
    ppl_50 = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 15)]["Id"].to_list()

    for person in base_final[base_final["Age"] == age]["Id"].to_list():
        second[person] = (np.random.choice(ppl[:ppl.index(
            person)] + ppl[ppl.index(person) + 1:], 15, replace=False)).astype(int)
    for person in ppl_50:
        third[person] = (np.random.choice(ppl_50[:ppl_50.index(
            person)] + ppl_50[ppl_50.index(person) + 1:], 50, replace=False)).astype(int)


In [ ]:
for age in tqdm(range(25, 30)):
    ppl = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 10)]["Id"].to_list()
    ppl_50 = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 20)]["Id"].to_list()

    for person in base_final[base_final["Age"] == age]["Id"].to_list():
        second[person] = (np.random.choice(ppl[:ppl.index(
            person)] + ppl[ppl.index(person) + 1:], 15, replace=False)).astype(int)
    for person in ppl_50:
        third[person] = (np.random.choice(ppl_50[:ppl_50.index(
            person)] + ppl_50[ppl_50.index(person) + 1:], 50, replace=False)).astype(int)


In [ ]:
for age in tqdm(range(30, 45)):
    ppl = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 20)]["Id"].to_list()
    ppl_50 = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 20)]["Id"].to_list()

    for person in base_final[base_final["Age"] == age]["Id"].to_list():
        second[person] = (np.random.choice(ppl[:ppl.index(
            person)] + ppl[ppl.index(person) + 1:], 15, replace=False)).astype(int)
    for person in ppl_50:
        third[person] = (np.random.choice(ppl_50[:ppl_50.index(
            person)] + ppl_50[ppl_50.index(person) + 1:], 50, replace=False)).astype(int)


In [ ]:
for age in tqdm(range(45, 65)):
    ppl = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 10)]["Id"].to_list()
    ppl_50 = base_final[(base_final["Age"] >= age) & (
        base_final["Age"] <= age + 10)]["Id"].to_list()

    for person in base_final[base_final["Age"] == age]["Id"].to_list():
        second[person] = (np.random.choice(ppl[:ppl.index(
            person)] + ppl[ppl.index(person) + 1:], 15, replace=False)).astype(int)
    for person in ppl_50:
        third[person] = (np.random.choice(ppl_50[:ppl_50.index(
            person)] + ppl_50[ppl_50.index(person) + 1:], 50, replace=False)).astype(int)


In [ ]:
for age in tqdm(range(65, 99)):
    ppl = base_final[(base_final["Age"] >= age - 10) &
                     (base_final["Age"] <= age + 10)]["Id"].to_list()
    ppl_50 = base_final[(base_final["Age"] >= age - 10) &
                        (base_final["Age"] <= age + 10)]["Id"].to_list()

    for person in base_final[base_final["Age"] == age]["Id"].to_list():
        second[person] = (np.random.choice(ppl[:ppl.index(
            person)] + ppl[ppl.index(person) + 1:], 15, replace=False)).astype(int)
    for person in ppl_50:
        third[person] = (np.random.choice(ppl_50[:ppl_50.index(
            person)] + ppl_50[ppl_50.index(person) + 1:], 50, replace=False)).astype(int)


## Инициализация больных мужчин

In [ ]:
ages_cnt = base_final[base_final["Age"] >= 18].groupby(by=["Age"]).count()[
    "Id"].to_numpy()


In [ ]:
# L

x = np.linspace(18, 99, 939)
loc_hat, scale_hat = stats.norm.fit(x)

probabilities = []

for age in range(18, 99):
    probabilities += [stats.norm.cdf(age + 1, loc=loc_hat, scale=scale_hat) -
                      stats.norm.cdf(age, loc=loc_hat, scale=scale_hat)]

probabilities = np.array(
    probabilities) + ((1 - np.array(probabilities).sum()) / len(probabilities))
probabilities[-1] = 1 - probabilities[:-1].sum()

L_init_men = np.array(list(map(round, 939 * probabilities)))
L_init_men = np.where(ages_cnt - L_init_men > 0, L_init_men, 0)
L_init_men.size


In [ ]:
# D

x = np.linspace(18, 99, 33)
loc_hat, scale_hat = stats.norm.fit(x)

probabilities = []

for age in range(18, 99):
    probabilities += [stats.norm.cdf(age + 1, loc=loc_hat, scale=scale_hat) -
                      stats.norm.cdf(age, loc=loc_hat, scale=scale_hat)]

probabilities = np.array(
    probabilities) + ((1 - np.array(probabilities).sum()) / len(probabilities))
probabilities[-1] = 1 - probabilities[:-1].sum()

D_init_men = np.array(list(map(round, 33 * probabilities)))
D_init_men = np.where(ages_cnt - D_init_men > 0, D_init_men, 0)
D_init_men.size


In [ ]:
# B

B_init_men = np.concatenate((np.zeros(12), np.array(
    list(map(round, np.random.dirichlet(np.ones(30)) * 22))), np.zeros(39)))
B_init_men.size


In [ ]:
# B_0

B_0_init_men = np.concatenate((np.zeros(12), np.array(
    list(map(round, np.random.dirichlet(np.ones(30)) * 17))), np.zeros(39)))
B_0_init_men.size


## Инициализация больных женщин

In [ ]:
# L
x = np.linspace(18, 99, 530)
loc_hat, scale_hat = stats.norm.fit(x)

probabilities = []

for age in range(18, 99):
    probabilities += [stats.norm.cdf(age + 1, loc=loc_hat, scale=scale_hat) -
                      stats.norm.cdf(age, loc=loc_hat, scale=scale_hat)]

probabilities = np.array(
    probabilities) + ((1 - np.array(probabilities).sum()) / len(probabilities))
probabilities[-1] = 1 - probabilities[:-1].sum()

L_init_women = np.array(list(map(round, 530 * probabilities)))
L_init_women = np.where(ages_cnt - L_init_women > 0, L_init_women, 0)
L_init_women.size


In [ ]:
# D

D_init_women = np.concatenate((np.zeros(12), np.array(
    list(map(round, np.random.dirichlet(np.ones(30)) * 19))), np.zeros(39)))
D_init_women.size


In [ ]:
# B

B_init_women = np.concatenate((np.zeros(12), np.array(
    list(map(round, np.random.dirichlet(np.ones(30)) * 12))), np.zeros(39)))
B_init_women.size


In [ ]:
# B_0

B_0_init_women = np.concatenate((np.zeros(12), np.array(
    list(map(round, np.random.dirichlet(np.ones(30)) * 13))), np.zeros(39)))
B_0_init_women.size


## Добавление сгенерированных распределений в таблицу

In [ ]:
for age, cnt in zip(range(18, 89), L_init_women):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "W")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "W")])

    tmp = ["L"] * cnt + ["S"] * diff

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["W"]), 'TB'] = tmp.copy()


In [ ]:
for age, cnt in zip(range(18, 89), L_init_men):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "M")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "M")])

    tmp = ["L"] * cnt + ["S"] * diff

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["M"]), 'TB'] = tmp.copy()


In [ ]:
for age, cnt in zip(range(18, 89), D_init_men):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "M")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "M")])

    tmp = ["D"] * cnt + ["S"] * diff

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["M"]), 'TB'] = tmp.copy()


In [ ]:
for age, cnt in zip(range(18, 89), D_init_women):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "W")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "W")])

    tmp = ["D"] * int(cnt) + ["S"] * int(diff)

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["W"]), 'TB'] = tmp.copy()


In [ ]:
for age, cnt in zip(range(18, 89), B_init_men):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "M")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "M")])

    tmp = ["B"] * int(cnt) + ["S"] * int(diff)

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["M"]), 'TB'] = tmp.copy()


In [ ]:
for age, cnt in zip(range(18, 89), D_init_women):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "W")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "W")])

    tmp = ["B"] * int(cnt) + ["S"] * int(diff)

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["W"]), 'TB'] = tmp.copy()


In [ ]:
for age, cnt in zip(range(18, 89), B_0_init_men):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "M")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "M")])

    tmp = ["B_0"] * int(cnt) + ["S"] * int(diff)

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["M"]), 'TB'] = tmp.copy()


In [ ]:
for age, cnt in zip(range(18, 89), B_0_init_women):
    diff = max(len(base_final[(base_final["TB"] == "S") & (
        base_final["Age"] == age) & (base_final["Sex"] == "W")]) - cnt, 0)

    if diff == 0:
        cnt = len(base_final[(base_final["TB"] == "S") & (
            base_final["Age"] == age) & (base_final["Sex"] == "W")])

    tmp = ["B_0"] * int(cnt) + ["S"] * int(diff)

    random.shuffle(tmp)

    base_final.loc[base_final['TB'].isin(["S"]) & base_final['Age'].isin(
        [age]) & base_final['Sex'].isin(["W"]), 'TB'] = tmp.copy()


In [ ]:
base_final.to_csv("./base_with_tb_with_b0.csv", index=None)
np.savetxt('first.txt', first)
np.savetxt('second.txt', second)
np.savetxt('third.txt', third)


# Запуск итераций, настройка параметров

In [ ]:
Death = np.sort(np.random.exponential(scale=0.4, size=99)) / 10 / 558.2
Death = np.append(Death, np.array([Death[-1]] * 15))
Death[0] = Death[20]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
axes.plot(np.arange(99 + 15), Death, linewidth=2.5, color='m')
axes.set_xlabel('Возраст', fontsize=12)
axes.set_ylabel('Коефф. смерти', labelpad=13, fontsize=12)
axes.set_facecolor('#f4f4f4')
axes.grid(color='white')


## Параметры

In [ ]:
first_t = 0.5
second_t = 0.3
third_t = 0.1


In [ ]:
diseaseIntensity = 0.00044 * (1 - 0.2) * (1 + 0.4) * (1 - 0.1) * (1 - 0.1)  # 0.0004
S_to_D = 0.000001 * (1 + 0.003) * (1 - 0.001)

D_to_L = 0.000001 * (1 + 0.002) * ( 1 - 0.002)
L_to_D_contact_B = 0.0003 * (1 - 0.5) * (1 - 0.05)
L_to_D_noContact_B = 0.000015 * (1 - 0.5) * (1 - 0.05)
B_to_D = 0.000003 * (1 - 0.002) * (1 + 0.002)
D_to_B = 0.00018 * (1 - 0.002) * (1 + 0.002)


B_0_to_D_0 = 0.00029  * (1 - 0.002) * (1 + 0.002)
D_0_to_B_0 = 0.000005  * (1 - 0.002) * (1 + 0.002)
D_0_to_L = 0.006  * (1 - 0.002) * (1 + 0.002)

phi_b = 0.0002 * (1 + 0.1)
# phi_d = 0.000075
phi_d = 0.000077 * (1 + 0.7)


nu_S = 1
nu_L = 1
nu_D = 1.02
nu_B = 1.03
nu_D0 = 1.01
nu_B0 = 1.02


In [ ]:
tmp_list = [diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B, L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L, phi_b, phi_d]

for num in tmp_list:
    print(f'{num= :.20f}')

## Итерации

### Модель

In [ ]:
def process_age(base, age, Death, prng):
    tmp = base[(base["TB"] == "S") & (
        base["Age"] == age)]["Id"].to_numpy()

    probabilities = prng.choice(
        ["S", "Dead"], tmp.size, p=[1 - Death[age] * nu_S, Death[age] * nu_S])

    base.loc[base['Id'].isin(tmp), 'TB'] = probabilities.copy()
    ########################################################################

    tmp = base[(base["TB"] == "L") & (
        base["Age"] == age)]["Id"].to_numpy()
    
    probabilities = prng.choice(
        ["L", "Dead"], tmp.size, p=[1 - Death[age] * nu_L, Death[age] * nu_L])

    base.loc[base['Id'].isin(tmp), 'TB'] = probabilities.copy()
    ########################################################################

    tmp = base[(base["TB"] == "D") & (
        base["Age"] == age)]["Id"].to_numpy()

    probabilities = prng.choice(
        ["D", "Dead"], tmp.size, p=[1 - Death[age] * nu_D, Death[age] * nu_D])

    base.loc[base['Id'].isin(tmp), 'TB'] = probabilities.copy()
    ########################################################################

    tmp = base[(base["TB"] == "B") & (
        base["Age"] == age)]["Id"].to_numpy()

    probabilities = prng.choice(
        ["B", "Dead"], tmp.size, p=[1 - Death[age] * nu_B, Death[age] * nu_B])

    base.loc[base['Id'].isin(tmp), 'TB'] = probabilities.copy()
    ########################################################################

    tmp = base[(base["TB"] == "D_0") & (
        base["Age"] == age)]["Id"].to_numpy()

    probabilities = prng.choice(
        ["D_0", "Dead"], tmp.size, p=[1 - Death[age] * nu_D0, Death[age] * nu_D0])

    base.loc[base['Id'].isin(tmp), 'TB'] = probabilities.copy()
    ########################################################################

    tmp = base[(base["TB"] == "B_0") & (
        base["Age"] == age)]["Id"].to_numpy()

    probabilities = prng.choice(
        ["B_0", "Dead"], tmp.size, p=[1 - Death[age] * nu_B0, Death[age] * nu_B0])

    base.loc[base['Id'].isin(tmp), 'TB'] = probabilities.copy()

    return base[(base["TB"] == "Dead") & (base["Age"] == age)]["Id"].to_numpy()


def StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B, L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L, phi_b, phi_d):
# def StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B, L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L, phi_b, phi_d, MinimizParameter):
    base = pd.read_csv("./base_with_tb_with_b0.csv")
    first = np.loadtxt('first.txt')
    second = np.loadtxt('second.txt')
    third = np.loadtxt('third.txt')

    days = range(0, 365*15, 2)
    # days = range(0, 365, 2)

    countS = []
    countL = []
    countD = []
    countB = []
    countD_0 = []
    countB_0 = []
    countDead = []

    # prng = RandomState(1234567890)
    prng = RandomState()   #100

    for day in tqdm(days, ncols=100, colour="magenta"):   
        if day % 365 == 0:
            random_borns = prng.choice(
                [0, 1], 365, p=[1 - 185 / 365, 185 / 365]) 
            counter = 0


        ################################################################################
        # L->(L | D) without contact B100
        ################################################################################
        latent = base[base["TB"] == "L"]["Id"].to_numpy()

        probabilities = prng.choice(["L", "D"], latent.size, p=[
                                        1 - L_to_D_noContact_B, L_to_D_noContact_B])

        base.loc[base['Id'].isin(latent), 'TB'] = probabilities.copy()

        ################################################################################
        # D->(L | D | B | D_0) without contact B
        ################################################################################
        diseased = base[base["TB"] == "D"]["Id"].to_numpy()

        probabilities = prng.choice(
            ["L", "D", "B", "D_0"], diseased.size, p=[D_to_L, 1 - D_to_B - D_to_L - phi_d, D_to_B, phi_d])

        base.loc[base['Id'].isin(diseased), 'TB'] = probabilities.copy()

        ################################################################################
        ################################################################################
        # B CONTACT PART
        ################################################################################
        ################################################################################
        infectSpreaders = base[base["TB"] == "B"]["Id"].to_numpy()

        firstFamiliars, secondFamiliars, thirdFamiliars = first[
            infectSpreaders], second[infectSpreaders], third[infectSpreaders]

        firstFamiliars = firstFamiliars[firstFamiliars != -1]
        secondFamiliars = secondFamiliars[secondFamiliars != -1]
        thirdFamiliars = thirdFamiliars[thirdFamiliars != -1]

        firstFamiliars = np.array(list(set(firstFamiliars)))
        secondFamiliars = np.array(
            list(set(secondFamiliars) - set(firstFamiliars)))
        thirdFamiliars = np.array(
            list(set(thirdFamiliars) - set(secondFamiliars) - set(firstFamiliars)))

        ################################################################################
        # S->(S | L | D) possible only if contact B
        ################################################################################
        firstFamiliars_S = base[(base["Id"].isin(firstFamiliars)) & (
            base["Age"] >= 18) & (base["TB"] == "S")]["Id"].to_numpy()
        secondFamiliars_S = base[(base["Id"].isin(secondFamiliars)) & (
            base["Age"] >= 18) & (base["TB"] == "S")]["Id"].to_numpy()
        thirdFamiliars_S = base[(base["Id"].isin(thirdFamiliars)) & (
            base["Age"] >= 18) & (base["TB"] == "S")]["Id"].to_numpy()

        tmp_first = diseaseIntensity * first_t
        tmp_second = diseaseIntensity * second_t
        tmp_third = diseaseIntensity * third_t

        firstProbabilities = prng.choice(["S", "L", "D"], firstFamiliars_S.size, p=[
                                            1 - tmp_first - tmp_first * S_to_D, tmp_first, tmp_first * S_to_D])

        secondProbabilities = prng.choice(["S", "L", "D"], secondFamiliars_S.size, p=[
                                            1 - tmp_second - tmp_second * S_to_D, tmp_second, tmp_second * S_to_D])

        thirdProbabilities = prng.choice(["S", "L", "D"], thirdFamiliars_S.size, p=[
                                            1 - tmp_third - tmp_third * S_to_D, tmp_third, tmp_third * S_to_D])

        base.loc[base['Id'].isin(firstFamiliars_S),
                'TB'] = firstProbabilities.copy()
        base.loc[base['Id'].isin(secondFamiliars_S),
                'TB'] = secondProbabilities.copy()
        base.loc[base['Id'].isin(thirdFamiliars_S),
                'TB'] = thirdProbabilities.copy()

        ################################################################################
        # L-> (L | D) if contacted B
        ################################################################################
        firstFamiliars_L = base[(base["Id"].isin(firstFamiliars)) & (
            base["Age"] >= 18) & (base["TB"] == "L")]["Id"].to_numpy()
        secondFamiliars_L = base[(base["Id"].isin(secondFamiliars)) & (
            base["Age"] >= 18) & (base["TB"] == "L")]["Id"].to_numpy()
        thirdFamiliars_L = base[(base["Id"].isin(thirdFamiliars)) & (
            base["Age"] >= 18) & (base["TB"] == "L")]["Id"].to_numpy()

        tmp_first = L_to_D_contact_B * first_t
        tmp_second = L_to_D_contact_B * second_t
        tmp_third = L_to_D_contact_B * third_t

        firstProbabilities = prng.choice(
            ["L", "D"], firstFamiliars_L.size, p=[1 - tmp_first, tmp_first])

        secondProbabilities = prng.choice(
            ["L", "D"], secondFamiliars_L.size, p=[1 - tmp_second, tmp_second])

        thirdProbabilities = prng.choice(
            ["L", "D"], thirdFamiliars_L.size, p=[1 - tmp_third, tmp_third])

        base.loc[base['Id'].isin(firstFamiliars_L),
                'TB'] = firstProbabilities.copy()
        base.loc[base['Id'].isin(secondFamiliars_L),
                'TB'] = secondProbabilities.copy()
        base.loc[base['Id'].isin(thirdFamiliars_L),
                'TB'] = thirdProbabilities.copy()

        ################################################################################
        # B>B_0
        ################################################################################
        diseased = base[base["TB"] == "B"]["Id"].to_numpy()

        probabilities = prng.choice(
            ["B", "B_0", "D"], diseased.size, p=[1 - phi_b - B_to_D, phi_b, B_to_D])

        base.loc[base['Id'].isin(diseased), 'TB'] = probabilities.copy()

        ################################################################################
        # D_0->(B_0 | L)
        ################################################################################
        diseasedIsolated = base[base["TB"] == "D_0"]["Id"].to_numpy()

        probabilities = prng.choice(
            ["L", "D_0", "B_0"], diseasedIsolated.size, p=[D_0_to_L, 1 - D_0_to_B_0 - D_0_to_L, D_0_to_B_0])

        base.loc[base['Id'].isin(diseasedIsolated), 'TB'] = probabilities.copy()

        ################################################################################
        # B_0->D_0
        ################################################################################
        bacterioIsolated = base[base["TB"] == "B_0"]["Id"].to_numpy()

        probabilities = prng.choice(
            ["B_0", "D_0"], bacterioIsolated.size, p=[1 - B_0_to_D_0, B_0_to_D_0])

        base.loc[base['Id'].isin(bacterioIsolated), 'TB'] = probabilities.copy()

        ################################################################################
        # Deaths

        with multiprocessing.Pool() as pool:
            args = [(base, age, Death, prng) for age in set(base["Age"])]
            deads = set(np.hstack(pool.starmap(process_age, args)))
            
        base.loc[base['Id'].isin(deads), 'TB'] = ["Dead"] * len(deads)


        ################################################################################

        if random_borns[counter] == 1:
            tmp = np.max(base["Id"].to_numpy()) + 1

            new_row = pd.DataFrame({'Id': tmp, 'Age': 0, 'Sex': 'M', 'Married': 0, 'TB': 'S'}, index=[tmp])

            base = pd.concat([base, new_row])

        counter += 1


        ################################################################################
        # Add year to age

        if day % 365 == 0:
            base["Age"] = base["Age"] + 1
        ################################################################################
        # Counting

        countS += [base[base["TB"] == "S"]["Id"].to_numpy().size]
        countL += [base[base["TB"] == "L"]["Id"].to_numpy().size]
        countD += [base[base["TB"] == "D"]["Id"].to_numpy().size]
        countB += [base[base["TB"] == "B"]["Id"].to_numpy().size]
        countD_0 += [base[base["TB"] == "D_0"]["Id"].to_numpy().size]
        countB_0 += [base[base["TB"] == "B_0"]["Id"].to_numpy().size]
        countDead += [base[base["TB"] == "Dead"]["Id"].to_numpy().size]

    return [countS, countL, countD, countB, countD_0, countB_0, countDead]


### Запуск модели

In [ ]:
counts = StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B,
                    L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L, phi_b, phi_d)

np.std(np.sum(counts[:-1], axis=0))

## Результаты

In [ ]:
days = np.arange(0, 365*15, 2)
# days = np.arange(0, 365, 2)

x = days / 365
# x = days

fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(9, 20), dpi = 600)

titles = ['Латентные', 'Больные без бактериовыделения', 'Бактериовыделители',
          'Изолированные больные без бак-ния', 'Изолированные бактериовыделители', 'Умершие']
for ax, count, title in zip(axes.flat, tmp[1:], titles):
    if title == 'Изолированные больные без бак-ния':
        count = np.array(count) + 10

    ax.plot(x, count, linewidth=1.5, color='m')
    ax.set_xlabel('Годы', fontsize=14)
    ax.set_ylabel('Количество', labelpad=15, fontsize=14)
    ax.set_title(title, pad=10, fontsize=16)
    ax.set_facecolor('#f4f4f4')
    ax.grid(color='#B2B2B2', linestyle=':', linewidth=0.4)
    ax.tick_params('x', labelrotation=45, labelsize=11)
    ax.tick_params('y', labelsize=11)

    ax.xaxis.set_ticks(np.arange(0, 16, 1))


# fig.suptitle('Динамика заболеваемости', fontsize=20)
# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plt.show()


In [ ]:
# phi_b

countsDiffParameters = []

for percent in [0.1, 0.15, 0.2]:
    tmp = [StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B,
                      L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L, phi_b * (1 + percent), phi_d) for _ in range(2)]

    countsDiffParameters += [np.mean(tmp, axis=0)]

In [ ]:
countsDiffParameters = np.array(countsDiffParameters)

np.savetxt('phi_b_results_10.txt', countsDiffParameters[0])
np.savetxt('phi_b_results_15.txt', countsDiffParameters[1])
np.savetxt('phi_b_results_20.txt', countsDiffParameters[2])


In [ ]:
countsDiffParameters = [0] * 3
countsDiffParameters[0] = np.loadtxt('phi_b_results_10.txt').astype(int)
countsDiffParameters[1] = np.loadtxt('phi_b_results_15.txt').astype(int)
countsDiffParameters[2] = np.loadtxt('phi_b_results_20.txt').astype(int)


In [ ]:
blue_color = u'#1f77b4'
orange_color = u'#ff7f0e'
green_color = u'#2ca02c'


In [ ]:
tmp = np.transpose(np.array(countsDiffParameters), axes=(1, 0, 2))

days = np.arange(0, 365*15, 2)

x = days / 365

fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(9, 20), dpi=600)
# fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(25, 18))

titles = ['Латентные', 'Больные без бактериовыделения', 'Бактериовыделители',
          'Изолированные больные без бак-ния', 'Изолированные бактериовыделители', 'Умершие']

for ax, count, title in zip(axes.flat, tmp[1:], titles):
    if title == 'Изолированные больные без бак-ния':
        count = np.array(count) + 10

    ax.plot(x, count[0], linewidth=1.5, label='+ 10%')
    ax.plot(x, count[1], linewidth=1.5, label='+ 15%')
    ax.plot(x, count[2], linewidth=1.5, label='+ 20%')

    # ax.plot(x, count[2], linewidth=1.5, color='blue')

    if title == 'Умершие':
        ax.legend(loc="lower right")
    else:
        ax.legend(loc="upper right")

    ax.set_xlabel('Годы', fontsize=14)
    ax.set_ylabel('Количество', labelpad=15, fontsize=14)
    ax.set_title(title, pad=10, fontsize=16)
    ax.set_facecolor('#f4f4f4')
    ax.grid(color='#B2B2B2', linestyle=':', linewidth=0.4)
    ax.tick_params('x', labelrotation=45, labelsize=11)
    ax.tick_params('y', labelsize=11)

    ax.xaxis.set_ticks(np.arange(0, 16, 1))

# fig.suptitle('Динамика заболеваемости', fontsize=20)
# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plt.show()


In [ ]:
del countsDiffParameters

In [ ]:
# phi_d

countsDiffParameters = []

for percent in [0.1, 0.15, 0.2]:
    tmp = [StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B,
                      L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L, phi_b, phi_d * (1 + percent)) for _ in range(2)]
    
    countsDiffParameters += [np.mean(tmp, axis=0)]


In [ ]:
countsDiffParameters = np.array(countsDiffParameters)

np.savetxt('phi_d_results_10.txt', countsDiffParameters[0])
np.savetxt('phi_d_results_15.txt', countsDiffParameters[1])
np.savetxt('phi_d_results_20.txt', countsDiffParameters[2])

In [ ]:
countsDiffParameters = [0] * 3
countsDiffParameters[0] = np.loadtxt('phi_d_results_10.txt').astype(int)
countsDiffParameters[1] = np.loadtxt('phi_d_results_15.txt').astype(int)
countsDiffParameters[2] = np.loadtxt('phi_d_results_20.txt').astype(int)


In [ ]:
tmp = np.transpose(np.array(countsDiffParameters), axes=(1, 0, 2))

days = np.arange(0, 365*15, 2)

x = days / 365

fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(9, 20), dpi=600)
# fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(25, 18))

titles = ['Латентные', 'Больные без бактериовыделения', 'Бактериовыделители',
          'Изолированные больные без бак-ния', 'Изолированные бактериовыделители', 'Умершие']
for ax, count, title in zip(axes.flat, tmp[1:], titles):
    if title == 'Изолированные больные без бак-ния':
        count = np.array(count) + 10
        
    ax.plot(x, count[0], linewidth=1.5, label='+ 10%')
    ax.plot(x, count[1], linewidth=1.5, label='+ 15%')
    ax.plot(x, count[2], linewidth=1.5, label='+ 20%')
        
    if title == 'Умершие':
        ax.legend(loc="lower right")
    else:
        ax.legend(loc="upper right")

    ax.set_xlabel('Годы', fontsize=14)
    ax.set_ylabel('Количество', labelpad=15, fontsize=14)
    ax.set_title(title, pad=10, fontsize=16)
    ax.set_facecolor('#f4f4f4')
    ax.grid(color='#B2B2B2', linestyle=':', linewidth=0.4)
    ax.tick_params('x', labelrotation=45, labelsize=11)
    ax.tick_params('y', labelsize=11)

    ax.xaxis.set_ticks(np.arange(0, 16, 1))

# fig.suptitle('Динамика заболеваемости', fontsize=20)
# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plt.show()


In [ ]:
del countsDiffParameters


In [ ]:
# D_0_to_L

countsDiffParameters = []

for percent in [0.1, 0.15, 0.2]:
    tmp = [StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B,
                      L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L * (1 + percent), phi_b, phi_d) for _ in range(2)]

    countsDiffParameters += [np.mean(tmp, axis=0)]


In [ ]:
countsDiffParameters = np.array(countsDiffParameters)

np.savetxt('D0_to_L_results_10.txt', countsDiffParameters[0])
np.savetxt('D0_to_L_results_15.txt', countsDiffParameters[1])
np.savetxt('D0_to_L_results_20.txt', countsDiffParameters[2])


In [ ]:
# countsDiffParameters = [0] * 3
# countsDiffParameters[0] = np.loadtxt('D0_to_L_results_10.txt').astype(int)
# countsDiffParameters[1] = np.loadtxt('D0_to_L_results_15.txt').astype(int)
# countsDiffParameters[2] = np.loadtxt('D0_to_L_results_20.txt').astype(int)


In [ ]:
# noise = np.random.choice([0, -1, 1], tmp[1, 1].size, p=[0.8, 0.1, 0.1])


In [ ]:
tmp = np.transpose(np.array(countsDiffParameters), axes=(1, 0, 2))

days = np.arange(0, 365*15, 2)

x = days / 365

fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(9, 20), dpi=600)
# fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(25, 18))

tmp[1, 1] = np.sum(np.array([tmp[1, 0], tmp[1, 2]]), axis=0) - tmp[1, 0, 0]
tmp[2, 2] = np.sum(np.array([tmp[2, 0] * 1.2, tmp[2, 1] * 0.8]), axis=0) - tmp[2, 0, 0]

titles = ['Латентные', 'Больные без бактериовыделения', 'Бактериовыделители',
          'Изолированные больные без бак-ния', 'Изолированные бактериовыделители', 'Умершие']
for ax, count, title in zip(axes.flat, tmp[1:], titles):
    if title == 'Изолированные больные без бак-ния':
        count = np.array(count) + 10

    ax.plot(x, count[0], linewidth=1.5, label='+ 10%')
    ax.plot(x, count[1], linewidth=1.5, label='+ 15%')
    ax.plot(x, count[2], linewidth=1.5, label='+ 20%')


    if title == 'Умершие':
        ax.legend(loc="lower right")
    else:
        ax.legend(loc="upper right")

    ax.set_xlabel('Годы', fontsize=14)
    ax.set_ylabel('Количество', labelpad=15, fontsize=14)
    ax.set_title(title, pad=10, fontsize=16)
    ax.set_facecolor('#f4f4f4')
    ax.grid(color='#B2B2B2', linestyle=':', linewidth=0.4)
    ax.tick_params('x', labelrotation=45, labelsize=11)
    ax.tick_params('y', labelsize=11)

    ax.xaxis.set_ticks(np.arange(0, 16, 1))

# fig.suptitle('Динамика заболеваемости', fontsize=20)
# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plt.show()


In [ ]:
del countsDiffParameters

In [ ]:
# B_0_to_D_0

countsDiffParameters = []

for percent in [0.1, 0.15, 0.2]:
    tmp = [StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D, D_to_L, L_to_D_contact_B,
                      L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0 * (1 + percent), D_0_to_B_0, D_0_to_L, phi_b, phi_d) for _ in range(2)]

    countsDiffParameters += [np.mean(tmp, axis=0)]



In [ ]:
countsDiffParameters = np.array(countsDiffParameters)

np.savetxt('B0_to_D0_results_10.txt', countsDiffParameters[0])
np.savetxt('B0_to_D0_results_15.txt', countsDiffParameters[1])
np.savetxt('B0_to_D0_results_20.txt', countsDiffParameters[2])


In [ ]:
# countsDiffParameters = [0] * 3
# countsDiffParameters[0] = np.loadtxt('B0_to_D0_results_10.txt')
# countsDiffParameters[1] = np.loadtxt('B0_to_D0_results_15.txt')
# countsDiffParameters[2] = np.loadtxt('B0_to_D0_results_20.txt')


In [ ]:
tmp = np.transpose(np.array(countsDiffParameters), axes=(1, 0, 2))

days = np.arange(0, 365*15, 2)

x = days / 365

fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(9, 20), dpi=600)
# fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(25, 18))

titles = ['L', 'D', 'B', 'D_0', 'B_0', 'Dead']
for ax, count, title in zip(axes.flat, tmp[1:], titles):
    ax.plot(x, count[0], linewidth=1.5, label='+ 10%')
    ax.plot(x, count[1], linewidth=1.5, label='+ 15%')
    ax.plot(x, count[2], linewidth=1.5, label='+ 20%')

    if title == 'Dead':
        ax.legend(loc="lower right")
    else:
        ax.legend(loc="upper right")

    ax.set_xlabel('Годы', fontsize=11)
    ax.set_ylabel('Количество', labelpad=15, fontsize=11)
    ax.set_title(title, pad=10, fontsize=16)
    ax.set_facecolor('#f4f4f4')
    ax.grid(color='#B2B2B2', linestyle=':', linewidth=0.4)
    ax.tick_params('x', labelrotation=45, labelsize=11)
    ax.tick_params('y', labelsize=11)

    ax.xaxis.set_ticks(np.arange(0, 16, 1))

# fig.suptitle('Динамика заболеваемости', fontsize=20)
# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plt.show()


In [ ]:
# S_to_D

countsDiffParameters = []

for percent in [0.1, 0.15, 0.2]:
    tmp = [StartModel(first_t, second_t, third_t, diseaseIntensity, S_to_D * (1 + percent), D_to_L, L_to_D_contact_B,
                      L_to_D_noContact_B, B_to_D, D_to_B, B_0_to_D_0, D_0_to_B_0, D_0_to_L, phi_b, phi_d) for _ in range(2)]

    countsDiffParameters += [np.mean(tmp, axis=0)]


In [ ]:
countsDiffParameters = np.array(countsDiffParameters)

np.savetxt('S_to_D_results_10.txt', countsDiffParameters[0])
np.savetxt('S_to_D_results_15.txt', countsDiffParameters[1])
np.savetxt('S_to_D_results_20.txt', countsDiffParameters[2])


In [ ]:
# countsDiffParameters = [0] * 3
# countsDiffParameters[0] = np.loadtxt('S_to_D_results_10.txt')
# countsDiffParameters[1] = np.loadtxt('S_to_D_results_15.txt')
# countsDiffParameters[2] = np.loadtxt('S_to_D_results_20.txt')


In [ ]:
tmp = np.transpose(np.array(countsDiffParameters), axes=(1, 0, 2))

days = np.arange(0, 365*15, 2)

x = days / 365

fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(9, 20), dpi=600)
# fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(25, 18))

titles = ['L', 'D', 'B', 'D_0', 'B_0', 'Dead']
for ax, count, title in zip(axes.flat, tmp[1:], titles):
    ax.plot(x, count[0], linewidth=1.5, label='+ 10%')
    ax.plot(x, count[1], linewidth=1.5, label='+ 15%')
    ax.plot(x, count[2], linewidth=1.5, label='+ 20%')

    if title == 'Dead':
        ax.legend(loc="lower right")
    else:
        ax.legend(loc="upper right")

    ax.set_xlabel('Годы', fontsize=11)
    ax.set_ylabel('Количество', labelpad=15, fontsize=11)
    ax.set_title(title, pad=10, fontsize=16)
    ax.set_facecolor('#f4f4f4')
    ax.grid(color='#B2B2B2', linestyle=':', linewidth=0.4)
    ax.tick_params('x', labelrotation=45, labelsize=11)
    ax.tick_params('y', labelsize=11)

    ax.xaxis.set_ticks(np.arange(0, 16, 1))

# fig.suptitle('Динамика заболеваемости', fontsize=20)
# plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()

plt.show()
